In [ ]:
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.tools import tool
from langchain.agents import create_openai_functions_agent
from langchain.tools.render import format_tool_to_openai_function
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages.base import BaseMessage
from langchain_core.messages.function import FunctionMessage
from langchain_core.messages import HumanMessage, SystemMessage
import operator
import functools


In [ ]:
load_dotenv(find_dotenv())

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    next_node: str

In [20]:

def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    functions = [format_tool_to_openai_function(t) for t in tools]
    prompt = ChatPromptTemplate.from_messages([
        (
            "System",
            "You are a helpful AI Assistant, collaborating with other assistants."
            "Use the provided tools to progress towards answering the question."
            "If you are unable to answer, that's okay, another agent with different tools"
            "will help you where you left off. Execute what you can to make progress."
            "If you or any of the other assistants have the final answer or deliverable,"
            "prefix your response with FINAL ANSWER so the team knows to stop."
            "You have access to the following tools: {tool_names}.\n{system_message}",
        ),
        MessagesPlaceholder(variable_name="messages")
    ])
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_functions(functions)

In [21]:

tavily_tool = TavilySearchResults(max_results=5)

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str
    next_node: str

In [26]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    if isinstance(result, FunctionMessage):
        pass
    else:
        result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages":[result],
        "sender": name
    }

In [ ]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.1")
response = llm.invoke("The first man on the moon was ...")
print(response)